In [1]:
from itertools import groupby
import tensorflow as tf
import random
import torch
import numpy as np
####### READING THE FASTA CHAIN DATA FROM THE FILE ############

def fasta_iter(name):
    fh = open(name)

    faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))

    for header in faiter:
        # drop the ">"
        headerStr = header.__next__()[1:].strip()

        seq = "".join(s.strip() for s in faiter.__next__())

        yield (headerStr, seq)

data=fasta_iter("Ecoli_prom.fa.txt")
training_data=[]

######## CONVERTING DNA SEQUENCE INTO ONE HOT ENCODING ##########

for chain in data:
    headerStr, seq = chain
    #print(seq)
    temp=[]
    for i in seq:
        if(i=='A'):
            temp.append([1,0,0,0])
        if(i=='T'):
            temp.append([0,1,0,0])
        if(i=='G'):
            temp.append([0,0,1,0])
        if(i=='C'):
            temp.append([0,0,0,1])
    
    training_data.append(temp)
       
    
    
data=fasta_iter("Ecoli_non_prom.fa.txt")
for chain in data:
    headerStr, seq = chain
    temp=[]
    for i in seq:
        if(i=='A'):
            temp.append([1,0,0,0])
        if(i=='T'):
            temp.append([0,1,0,0])
        if(i=='G'):
            temp.append([0,0,1,0])
        if(i=='C'):
            temp.append([0,0,0,1])
        
    training_data.append(temp)

#training_data

In [2]:
########### CREATING LABELS FOR THE OUTPUT ###########
labels=[]
for i in range(839):
    labels.append(1)
for i in range(3000):
    labels.append(0)

#### SHUFFLE THE DATA ########    

comb=list(zip(training_data,labels))
import time
random.seed(time.time())

random.shuffle(comb)
training_data,labels=zip(*comb)


print(np.shape(training_data))
print(np.shape(labels))

training_data=np.reshape(training_data,(3839,81*4))

print(np.shape(training_data))


(3839, 81, 4)
(3839,)
(3839, 324)


In [3]:
####### SPLITTING DATA INTO TRAIN,VALIDATION AND TEST ############
X_train=training_data[:3000]
Y_train=labels[:3000]

X_valid=training_data[3000:3400]
Y_valid=labels[3000:3400]

X_test=training_data[3400:]
Y_test=labels[3400:]

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier


/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
parameters={
    "booster":'gbtree',
    'learning_rate':0.03,
    "n_estimators":1000,
    "max_depth":3,
    "seed":1,
    "min_child_weight":3,
    
    "colsample_bylevel":0.8,
    "colsample_bytree":0.7,
    

}

In [6]:
model = XGBClassifier(**parameters)
model.fit(X_train, Y_train)
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.8,
       colsample_bytree=0.7, gamma=0, learning_rate=0.03, max_delta_step=0,
       max_depth=3, min_child_weight=3, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1, silent=True,
       subsample=1)

In [7]:
# make predictions for test data

y_pred1 = model.predict(X_train)
predictions1 = [round(value) for value in y_pred1]
y_pred = model.predict(X_valid)
predictions = [round(value) for value in y_pred]

/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
# evaluate predictions
accuracy = accuracy_score(Y_train, predictions1)
print("Train Accuracy: %.2f%%" % (accuracy * 100.0))
accuracy = accuracy_score(Y_valid, predictions)
print("Validation Accuracy: %.2f%%" % (accuracy * 100.0))

Train Accuracy: 99.10%
Validation Accuracy: 92.25%


In [9]:
y_pred2 = model.predict(X_test)
predictions2 = [round(value) for value in y_pred2]
accuracy = accuracy_score(Y_test, predictions2)
print("Test Accuracy: %.2f%%" % (accuracy * 100.0))

Test Accuracy: 90.21%


/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
